In [1]:
import dew_gwdata as gd
import pandas as pd

In [2]:
db = gd.sageodata()

In [14]:
sql = """
SELECT dh.drillhole_no                             AS dh_no,
       dh.unit_no                                  AS unit_long,
       To_char(dh.map_100000_no)
       || '-'
       || To_char(dh.dh_seq_no)                    AS unit_hyphen,
       Trim(To_char(dh.obs_well_plan_code))
       || Trim(To_char(dh.obs_well_seq_no, '000')) AS obs_no,
       dh.dh_name,
       dh.amg_easting                              AS easting,
       dh.amg_northing                             AS northing,
       dh.amg_zone                                 AS zone,
       dh.neg_lat_deg_real                         AS latitude,
       dh.long_deg_real                            AS longitude,
       summ.aq_subaq                               AS current_aquifer,
       aqmon.constrn_date                          AS aquifer_mon_from,
       su.map_symbol
       || CASE
            WHEN hs_subint.hydro_subunit_code IS NOT NULL THEN '('
                                                               ||
            hs_subint.hydro_subunit_code
                                                               || ')'
            ELSE ''
          END                                      AS aquifer
FROM   dhdb.dd_dh_aquifer_mon_vw aqmon
       inner join dhdb.dd_drillhole_vw dh
               ON aqmon.drillhole_no = dh.drillhole_no
       inner join dhdb.dd_drillhole_summary_vw summ
               ON aqmon.drillhole_no = summ.drillhole_no
       inner join dhdb.st_strat_unit_vw su
               ON aqmon.strat_unit_no = su.strat_unit_no
       left outer join (SELECT DISTINCT strat_unit_no,
                                        hydro_subunit_code
                        FROM   dhdb.wa_hydrostrat_subint_vw) hs_subint
                    ON aqmon.strat_unit_no = hs_subint.strat_unit_no
                       AND aqmon.hydro_subunit_code =
                           hs_subint.hydro_subunit_code
       left outer join dhdb.wa_hydrostrat_subunit_vw hs_subunit
                    ON hs_subint.strat_unit_no = hs_subunit.strat_unit_no
                       AND hs_subint.hydro_subunit_code =
                           hs_subunit.hydro_subunit_code
WHERE  aqmon.drillhole_no IN (SELECT a.drillhole_no
                              FROM   dhdb.dd_dh_aquifer_mon_vw a
                              inner join dhdb.st_strat_unit_vw stu ON a.strat_unit_no = stu.strat_unit_no
                              left outer join (SELECT DISTINCT strat_unit_no, hydro_subunit_code FROM dhdb.wa_hydrostrat_subint_vw) hss
                                  ON a.strat_unit_no = hss.strat_unit_no AND a.hydro_subunit_code = hss.hydro_subunit_code
                              where (stu.map_symbol
       || CASE
            WHEN hss.hydro_subunit_code IS NOT NULL THEN '('
                                                               ||
            hss.hydro_subunit_code
                                                               || ')'
            ELSE ''
          END) in {AQUIFER}
                              )
ORDER  BY dh_no,
          aquifer_mon_from
"""
df = db.query(sql)
# dff = df.pipe(gd.cleanup_columns)
df
# df[df.obs_no == 'SQR076']

,dh_no,unit_long,unit_hyphen,obs_no,dh_name,easting,northing,zone,latitude,longitude,current_aquifer,aquifer_mon_from,aquifer
0,7700,593000028,5930-28,TAA059,None,502201.73,6287581.52,53,-33.551403,135.023718,Tbe,1969-03-27,Qpcb
1,7700,593000028,5930-28,TAA059,None,502201.73,6287581.52,53,-33.551403,135.023718,Tbe,1989-09-07,Tbe
2,7703,593000031,5930-31,TAA021,None,502338.24,6288123.82,53,-33.546511,135.025187,Qpcb+Tbe,1966-10-14,Qpcb
3,7703,593000031,5930-31,TAA021,None,502338.24,6288123.82,53,-33.546511,135.025187,Qpcb+Tbe,2010-11-15,Tbe
4,7858,593000186,5930-186,HUD068,None,510046.37,6283836.60,53,-33.585135,135.108265,Qpcb+Tbe,2010-10-29,Qpcb
5,7858,593000186,5930-186,HUD068,None,510046.37,6283836.60,53,-33.585135,135.108265,Qpcb+Tbe,2010-10-29,Tbe
6,7945,593000273,5930-273,HUD021,None,514466.26,6264986.52,53,-33.755100,135.156203,Tbe,2010-11-04,Tbe
7,7947,593000275,5930-275,HUD023,None,517361.29,6264965.84,53,-33.755243,135.187463,Qpcb+Tbe,1987-04-14,Qpcb
8,7947,593000275,5930-275,HUD023,None,517361.29,6264965.84,53,-33.755243,135.187463,Qpcb+Tbe,2010-11-04,Tbe
9,8015,593000343,5930-343,WAY054,None,519360.69,6256481.50,53,-33.831730,135.209238,Tbe,1967-07-14,Qpcb


In [ ]:
(su.map_symbol
       || CASE
            WHEN hs_subint.hydro_subunit_code IS NOT NULL THEN '('
                                                               ||
            hs_subint.hydro_subunit_code
                                                               || ')'
            ELSE ''
          END)